This notebook aims to provide a gentle introduction of Shannon's entropy and mutual information.

**Please do not distribute this material without consent of the author.
@Copyright 2025
Peishi Jiang**

# What is information theory?
Information theory is a mathematical method to quantify the nonlinear dependencies in a multivariate system. It originates from electronic engineering to study the storage, quantification, and communication of information. Claude Shannon pioneered this field and proposed the concept of Shannon's entropy in the 1940s. Since then, this mathematical branch has gone through great development and applications in various field, including but not limited to, electronic engineering, mathematics, computer science, phyics, neurobiology, and earth science/hydrology.


# Entropy
We use Shannon's entropy here, denoted of $H$. Given a variable $X$, $H$ quantifies the overall uncertainty (or variability) of $X$ as:
\begin{align}
 H(X) &= - \sum_{x \in X} p(x) \log(p(x)) \tag{1},
\end{align}
where $p(x)$ is the discretized probability of the realization of $X=x$.

For two variables $X$ and $Y$, their overall uncertainty can be calculated through the joint entropy as below:
\begin{align}
 H(X,Y) &= - \sum_{x \in X} \sum_{y \in Y} p(x,y) \log(p(x,y)) \tag{2}.
\end{align}

## Entropy's properties
Shannon's entropy has the following nice properties (**Prove them!**):
1. Nonnegativity: $H \geq 0$. In other words, the entropy is always nonnegative.
2. Chain rule: $H(X,Y) = H(X) + H(Y|X)$, where $H(Y|X)=-\sum_{x \in X} \sum_{y \in Y} p(x,y) \log(p(y|x))$ is the conditional entropy. This property states that the joint uncertainty of $X$ and $Y$ is the sum of $X$'s uncertainty ($H(X)$) and the remaining uncertainty of $Y$ given the knowledge of $X$ ($H(Y|X)$).
3. $H(X|Y) \leq H(X)$.

## Maximum and minimum entropy (univariate case)
Now, let's move onto the calculation of the entropy $H$ and see under what conditions it reaches maximum/minimum.

OK, say a variable $X$ has two states $X=a$ and $X=b$, with probabilities $P(X=a)$ and $P(X=b)$, respectively, such that $P(X=a) + P(X=b) = 1$. Instead of mannually calculating it (which you can!), we are going to use Python programming to compute it.

Let's first define a function to calculate the entropy --

In [11]:
import numpy as np
def computeEntropy(pdf, dims=None, base=np.e):
    """
    Compute the entropy along dims
    pdf: probability mass distribution with shape (Nx, Ny, Nz, ...)
    dims: the dimension indices
    """
    shape = pdf.shape
    
    # Get the number of variables/dimensions
    D = len(shape)

    # Get the indices of all dimensions/variabiels
    all_dims  = set(range(D))

    # Compute the marginal PDF
    if dims is None:
        pdf_dim = pdf
    elif all_dims == set(dims):
        pdf_dim = pdf
    else:
        pdf_dim = pdf.sum(axis=tuple(all_dims-set(dims)))

    # Compute the entropy
    log_pdf_dim = np.ma.filled(np.log(np.ma.masked_equal(pdf_dim, 0)), 0)
    return -np.sum(pdf*log_pdf_dim / np.log(base))

# def computeEntropy(pdf, base=np.e):
#     """
#     Compute the entropy based on pdf
#     pdf: a numpy array of probability such that sum(pdf)=1
#     base: the logarithm base
#     """
#     log_pdf = np.ma.filled(np.log(np.ma.masked_equal(pdf, 0)), 0)
#     return -np.sum(pdf*log_pdf / np.log(base))


Scenario 1: X is determinant such that $P(X=a)=1$ and $P(X=b)=0$. What is $H(X)$?

In [12]:
P1 = np.array([1, 0])
H1 = computeEntropy(P1)
print(f"H1: {H1} nats")

H1: -0.0 nats


/var/folders/b7/q6wyxw8x1_s5rh1mqvfb8vsr0000gn/T/ipykernel_19027/1805999971.py:25: RuntimeWarning: divide by zero encountered in log
  log_pdf_dim = np.ma.filled(np.log(np.ma.masked_equal(pdf_dim, 0)), 0)


$H(X) = 0$ ! This means there is no variability or uncertainty associated with $X$, which totally makes sense since $X$ is deterministic. Let's consider two additional scenarios:

Scenario 2: $P(X=a)=0.1$ and $P(X=b)=0.9$

Scenario 3: $P(X=a)=0.5$ and $P(X=b)=0.5$

In [13]:
P2, P3 = np.array([0.1, 0.9]), np.array([0.5, 0.5])
H2 = computeEntropy(P2)
H3 = computeEntropy(P3)
print(f"H2: {H2} nats")
print(f"H3: {H3} nats")

H2: 0.3250829733914482 nats
H3: 0.6931471805599453 nats


Why does $H2 < H3$? 

Intuitively, this is because scenario 2 is more deterministic than scenario 3 since there is a much higher chance that $X=b$. On the other hand, scenario 3 gives equal probability to $X=a$ and $X=b$.

In fact, one can prove that the entropy is maximized when the distribution follows a uniform distribution! Note that scenario 3 is exactly a discretized uniform distribution with only two realizations. In a generalized discrete uniform distribution with $N$ realizations, $H = \log(\frac{1}{N})$.

## Maximum and minimum entropy (bivariate case)
The same philosophy is applicable to a bivariate case.

Now consider another variable $Y$ that takes on values $c$ and $d$. 

Scenario 4: The joint probabilty of $X$ and $Y$ are given by:
\begin{align}
 P(X=a,Y=c) &= 0.1 \notag\\
 P(X=b,Y=c) &= 0.2 \notag\\
 P(X=a,Y=d) &= 0.3 \notag\\
 P(X=b,Y=d) &= 0.4. \notag
\end{align}

Their joint entropy is --

In [21]:
P4 = np.array([[0.1, 0.2], [0.3, 0.4]])
H4 = computeEntropy(P4)
print(f"H4: {H4} nats")

H4: 1.2798542258336676 nats


Scenario 5: What if it follows uniform distribution? We are expecting that the joint entropy would be larger than $H4$.

In [22]:
P5 = np.array([[0.25, 0.25], [0.25, 0.25]])
H5 = computeEntropy(P5)
print(f"H5: {H5} nats")

H5: 1.3862943611198906 nats


## What's the units of entropy?
It depends on what base is used in the log! If it is a natural base (as what has been used in the previous cases), the units is "nats". If the base adopts 2, the units is "bits"!

Now, let's revisit the three scenarios.


In [16]:
base = 2.
H1 = computeEntropy(P1, base=2)
H2 = computeEntropy(P2, base=2)
H3 = computeEntropy(P3, base=2)
print(f"H1: {H1} bits")
print(f"H2: {H2} bits")
print(f"H3: {H3} bits")

H1: -0.0 bits
H2: 0.4689955935892812 bits
H3: 1.0 bits


/var/folders/b7/q6wyxw8x1_s5rh1mqvfb8vsr0000gn/T/ipykernel_19027/1805999971.py:25: RuntimeWarning: divide by zero encountered in log
  log_pdf_dim = np.ma.filled(np.log(np.ma.masked_equal(pdf_dim, 0)), 0)


Now, the results are different. Sometimes, bits are more intuitive for binary cases. Here, $H_3=1$ since $\log_2(1/2) = 1$.

## Differential entropy

We have covered Shannon's entropy applied to the discretized probability space. What about continuous probability distribution? Well, this is where differential entropy is applied (denoted as lower case $h$):
\begin{align}
 h(X) &= - \int_{x \in X} f(x) \log(f(x)) \mathrm{d}x\tag{3},
\end{align}
where $f$ is the probability density function of $X$.

Differential entropy is very useful when the functional form of $f$ is given and the analytical solutions of $h$ can be derived. Take the uniform distribution that are defined over $[a, b]$ as an example. The corresponding $h$ can be derived as:
\begin{align}
 h(X) = \log(b-a)\tag{4}.
\end{align}

For Gaussian distribution $f(x) = \frac{1}{\sqrt{2\pi \sigma^2}}e^{\frac{-x^2}{2\sigma^2}}$, the corresponding $h$ is given as:
\begin{align}
 h(X) = \frac{1}{2}\log(2\pi e \sigma^2)\tag{5}.
\end{align}

**Note that differential entropy can be negative!**

# Mutual information

Consider two random variables $X$ and $Y$ with a joint probability distribution $p(x,y)$. The mutual information $I(X;Y)$ quantifies the shared information or uncertainty between $X$ and $Y$, and it is given by:
\begin{align}
 I(X;Y) &= \sum_{x \in X} \sum_{y \in Y} p(x,y) \log(\frac{p(x,y)}{p(x)p(y)})\tag{6}.
\end{align}


## Mutual information's properties

While it might be a bit abstract at the first glance, Eq.(6) can in fact be converted into:
\begin{align}
 I(X;Y) &= H(Y) - H(Y|X)\tag{7a} \\
        &= H(X) - H(X|Y)\tag{7b} \\
        &= H(X) + H(Y) - H(X,Y)\tag{7c}
\end{align}
The above equations describes that $I(X;Y)$ is the difference between the entropy of one variable and the remaining uncertainty of this variable given the knowledge of the other other. As a result, $I(X;Y)$ is a symmetric metric, such that $I(X;Y)=I(Y;X)$.

**Properties of $I(X;Y)$**:
1. $I(X;Y) = H(Y) - H(Y|X) = H(X) - H(X|Y) = H(X) + H(Y) - H(X,Y)$.
2. $I(X;Y) = 0$ when $X$ and $Y$ are independent such that $P(X,Y) = P(X)P(Y)$.

Let's define a function that calculates $I(X;Y)$.


In [23]:
def computeMI(pdf, dims1, dims2, base=np.e):
    """
    Compute the mutual information given two sets of dimensions
    dims1 -- the first dimension indices
    dims2 -- the second dimension indices
    normalized -- whether to normalize the mutual information through dividing it by the output entropy
    """
    # Compute the entropies
    h12 = computeEntropy(pdf, dims1+dims2, base)
    h1  = computeEntropy(pdf, dims1, base)
    h2  = computeEntropy(pdf, dims2, base)

    # Compute the mutual information
    return h1 + h2 - h12


Let's revisit scenarios 4 and 5. What are the mutual information of $X$ and $Y$ for the two cases? Which is larger?

In [24]:
I1 = computeMI(P4, dims1=[0], dims2=[1])
I2 = computeMI(P5, dims1=[0], dims2=[1])

print(f"I1: {I1} nats")
print(f"I2: {I2} nats")

I1: 0.08875152926920249 nats
I2: 0.0 nats


Why is $I2$ (scenario 5) zero? This is because $X$ and $Y$ are independent in this uniform distribution! So, the second property of $I(X;Y)$ holds.

## Mutual information for differential entropy
The mutual information $I(X;Y)$ between two random variables with joint density $f(x,y)$ is defined as:
\begin{align}
 I(X;Y) = \int_{x\in X} \int_{y \in Y} f(x,y) \log\frac{f(x,y)}{f(x)f(y)}\mathrm{d}x\mathrm{d}y\tag{8}
\end{align}

Let $(X,Y) \sim \mathcal{N}(0,K)$, where
\begin{align}
 K = \begin{bmatrix} \sigma^2 & \rho \sigma^2 \\ \rho \sigma^2 & \sigma^2 \end{bmatrix}\notag.
\end{align}

We can show that
\begin{align}
 I(X;Y) = -\frac{1}{2}\log(1-\sigma^2)\tag{9}
\end{align}

# Real world application


In [ ]:
# Binning methods

In [ ]:
# Exploratory analysis


In [ ]:
# Homework: lagged mutual information


# Homework

## Entropy

**Ex1**: Given a variable $X_2$ that has four states $a,b,c,$ and $d$, what is its entropy if their probabilities are $P(X=a)=0$, $P(X=b)=0$, $P(X=c)=0$, and $P(X=d)=1$? Please use the natural number as the base.

**Ex2**: In **Ex1**, if their probability follows discretized uniform distribution, what is its entropy? Does it equal to $H_3$ in scenario 3? If not, why?

**Ex3**: Given two variables $X_3$ and $X_4$, each of which takes on five different realizations/values. Suppose their joint probability follows a discretized uniform distribution, what is the corresponding joint entropy?

**Ex4**: Please prove the chain rule of the entropy $H(X,Y) = H(X) + H(Y|X)$.

**Ex5**: Please prove Eq.(5) that $h(X) = \frac{1}{2}\log(2\pi e \sigma^2)$ when $f(x) = \frac{1}{\sqrt{2\pi \sigma^2}}e^{\frac{-x^2}{2\sigma^2}}$.

**Ex6**: TBD, related to the data used combined with the binning methods...

## Mutual information

**Ex7**: Prove $I(X;Y) = H(Y) - H(Y|X)$ in Eq.(7a).

**Ex8**: Prove $I(X;Y) = H(X) + H(Y) - H(X,Y)$ in Eq.(7c).

**Ex9**: Prove $I(X;Y) = 0$ when $X$ and $Y$ are independent such that $P(X,Y) = P(X)P(Y)$.

**Ex9**: Please write a function to calculate the conditional entropy $H(X|Y)$ and compute $H(X|Y)$ for the joint probability used in scenario 4.

**Ex10**: Prove $I(X;Y) = -\frac{1}{2}\log(1-\sigma^2)$ in Eq.(9).

# Further reading
- Shannon, C. E. (1948). A mathematical theory of communication. The Bell system technical journal, 27(3), 379-423.
- Cover, T., & Thomas, J. (2006). Chapter 2: Entropy, Relative Entropy, and Mutual Information. Elements of information theory (2nd., Vol. 2). New York: Wiley.
- Cover, T., & Thomas, J. (2006). Chapter 8: Differential Entropy. Elements of information theory (2nd., Vol. 2). New York: Wiley.
- Gong, W., Yang, D., Gupta, H. V., & Nearing, G. (2014). Estimating information entropy for hydrological data: One‐dimensional case. Water Resources Research, 50(6), 5003-5018.